In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset
import os
from dotenv import load_dotenv
from huggingface_hub import login
load_dotenv()
login(os.getenv("HF_ACCESS_TOKEN"))

In [2]:
df = pd.read_csv("../data/aes_dataset.csv")
df.head()

,question,reference_answer,answer,score,dataset,max_length1,normalized_score
0,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sumber tenaga, pemanis alami, menjaga sistem i...",27.0,analisis_essay,65,0.27
1,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sebagai sumber energi, pemanis alami, menjaga ...",21.0,analisis_essay,66,0.21
2,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,1. Sebagai energi. 2. Sebagai memperlancaar pe...,42.0,analisis_essay,76,0.42
3,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"untuk membuat kenyang, agar tidak lapar, agar ...",18.0,analisis_essay,67,0.18
4,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,Karbohidrat mempunyai peran penting untuk pros...,82.0,analisis_essay,105,0.82


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22406 entries, 0 to 22405
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          4859 non-null   object 
 1   reference_answer  22406 non-null  object 
 2   answer            22406 non-null  object 
 3   score             22406 non-null  float64
 4   dataset           22406 non-null  object 
 5   max_length1       22406 non-null  int64  
 6   normalized_score  22406 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.2+ MB


# Split dataset

In [6]:
subset_dataset = df['dataset'].unique()
subset_dataset

array(['analisis_essay', 'asap', 'cunlp', 'sag', 'sci', 'stita'],
      dtype=object)

In [8]:
splits = {}

for subset in subset_dataset:
    subset_df = df[df['dataset'] == subset]

    # split dataset (70:20:10)
    train, temp = train_test_split(subset_df, test_size=0.3, random_state=42)
    valid, test = train_test_split(temp, test_size=0.3, random_state=42)

    splits[subset] = {
        'train': train,
        'valid': valid,
        'test': test,
    }

In [9]:
train_df = pd.concat([splits[subset]['train'] for subset in subset_dataset])
valid_df = pd.concat([splits[subset]['valid'] for subset in subset_dataset])
test_df = pd.concat([splits[subset]['test'] for subset in subset_dataset])

## Convert Pandas DF to huggingface dataset

In [12]:
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

## Push Dataset to HuggingFace

In [ ]:
dataset_dict.push_to_hub("bwbayu/automatic_essay_scoring")

## Load Dataset From HF

In [ ]:
dataset = load_dataset("bwbayu/automatic_essay_scoring")
train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']